### Preparing HDFS
Using magic

Create input folder on HDFS if not exists

Copy from data from local

<span style="color:red"><em style=font-size:40px;>!</em>The following cells are basically a copy and paste of our assessment 1 submission

In [46]:
!pwd
! hadoop fs -rm -R /tmp/rs_input
! hadoop fs -mkdir -p  /tmp/rs_input
! hadoop fs -put   -p  ./../data-raw/Melbourne_housing_FULL.csv             /tmp/rs_input/raw.csv
! hadoop fs -ls        /tmp/rs_input/

/home/big-data-realestate-master/scripts

20/06/07 03:33:08 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


Deleted /tmp/rs_input


Found 1 items


-rwxrwxrwx   1 1000 staff    5018236 2020-06-04 09:20 /tmp/rs_input/raw.csv




In [47]:
//load raw into df
val df_raw = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/rs_input/raw.csv")

df_raw: org.apache.spark.sql.DataFrame = [Suburb: string, Address: string ... 19 more fields]


In [48]:
//only select columns we need now
var df_working= df_raw.select("Price",
                          "Method",
                          "Type",
                          "Distance",
                          "Rooms",
                          "Bathroom",
                          "Car",
                          "Landsize",
                          "Lattitude",
                          "Longtitude",    
                          "Suburb",
                          "Address",
                          "Date")


//add meaningful to column names
df_working = df_working.withColumnRenamed("Method","MethodOfSale")
    .withColumnRenamed("Distance","DistanceFromCBD")
    .withColumnRenamed("Type","PropertyType")
    .withColumnRenamed("Lattitude","Latitude")

df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [49]:
//when profiling there are a number of columns with a "#N/A" which need to be removed
df_working = df_working.filter($"DistanceFromCBD" =!= "#N/A")

df_working: org.apache.spark.sql.DataFrame = [Price: string, MethodOfSale: string ... 11 more fields]


In [50]:
//refactored to remove the for column loop
df_working = df_working.withColumn("Price",col("Price").cast("Double"))
    .withColumn("Rooms",col("Rooms").cast("Int"))
    .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
    .withColumn("Bathroom",col("Bathroom").cast("Int"))
    .withColumn("Car",col("Car").cast("Int"))
    .withColumn("Landsize",col("Landsize").cast("Double"))
    .withColumn("Latitude",col("Latitude").cast("Double"))
    .withColumn("Longtitude",col("Longtitude").cast("Double"))
    

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [51]:
//convert categorically values to ints
//make sure the categorical type is upper
df_working = df_working.withColumn("PropertyType", upper(col("PropertyType")))

df_working = df_working.withColumn("PropertyType",
when(col("PropertyType") === "H", "1")
.when(col("PropertyType") === "U", "2")
.when(col("PropertyType") ==="T", "3")
.otherwise("0"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [52]:
//convert categorically values to ints
//make sure the categorical type is upper
df_working = df_working.withColumn("MethodOfSale", upper(col("MethodOfSale")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [53]:
df_working = df_working.withColumn("MethodOfSale",
when(col("MethodOfSale") === "S", "1")
.when(col("MethodOfSale") === "SP", "2")
.when(col("MethodOfSale") === "PI", "3")
.when(col("MethodOfSale") === "PN", "4")
.when(col("MethodOfSale") === "SN", "5")
.when(col("MethodOfSale") === "VB", "6")
.when(col("MethodOfSale") === "W", "7")
.when(col("MethodOfSale") === "SA", "8")
.when(col("MethodOfSale") === "SS", "9")                                
.otherwise("0"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: string ... 11 more fields]


In [54]:
//cast categorical values to Ints
//Not strictly needed with the one hot encoding later
df_working = df_working.withColumn("PropertyType",col("PropertyType").cast("Int"))
    .withColumn("MethodOfSale",col("MethodOfSale").cast("Int"))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [55]:
// make first letter of suburb upper case
df_working= df_working.withColumn("Suburb", initcap(col("Suburb")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [56]:
//split address on Street name and Suffix
df_working = df_working.withColumn("StreetName",split(col("Address")," ").getItem(1)).
    withColumn("StreetSuffix",split(col("Address")," ").getItem(2)).drop("Address")

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [57]:
//fix "The Parade, The *** adddresses"
df_working = df_working.withColumn("StreetName",
when(col("StreetName").like("The"), concat(lit("The "),col("StreetSuffix")))
.otherwise(col("StreetName")))

//remove the street suffix
df_working = df_working.withColumn("StreetSuffix",
when(col("StreetName").contains("The"), lit(""))
.otherwise(col("StreetSuffix")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]
df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [58]:
//Rebuild street name from cleaned tokens
//this is approach was due to the legacy code we already had tokenizing the address so i just rejoined them
//one could write a regex replace to remove the street numbers unit numbers etc. bu i know these columns are clean
df_working = df_working.withColumn("StreetName", concat(col("StreetName"), lit(" "),col("StreetSuffix"))).drop("StreetSuffix")

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [59]:
// make first letter of Street upper case
df_working= df_working.withColumn("StreetName", initcap(col("StreetName")))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [60]:
//drop all properties with land area less than 12 sqm 
df_working = df_working.filter(!($"Landsize"<12))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [61]:
//drop rows where type = h and landsize < 50 sqm
df_working = df_working.filter(!($"Landsize"<50 && $"PropertyType"===1))

df_working: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [62]:
val df_not_null = df_working.na.drop()

df_not_null: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [63]:
df_not_null.printSchema()

root
 |-- Price: double (nullable = true)
 |-- MethodOfSale: integer (nullable = true)
 |-- PropertyType: integer (nullable = true)
 |-- DistanceFromCBD: double (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Car: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- StreetName: string (nullable = true)



In [64]:
! hadoop fs -rm -R /tmp/output
! hadoop fs -mkdir -p /tmp/output

20/06/07 03:33:20 INFO fs.TrashPolicyDefault: Namenode trash configuration: Deletion interval = 0 minutes, Emptier interval = 0 minutes.


Deleted /tmp/output




In [65]:
//this coalesce and write out isn't necessary if doing e2e
val df_output = df_not_null.coalesce(1)
   .write
   .format("csv")
   .option("header","true")
   .mode("overwrite").option("sep",",")
   .save("hdfs://localhost:9000/tmp/output")

df_output: Unit = ()


In [66]:
! hadoop fs -mkdir -p /tmp/output
! rm ./../data-clean/cleanMelbourneData.csv
! hadoop fs -copyToLocal /tmp/output/\*.csv ./../data-clean/cleanMelbourneData.csv

##  <span style="color:red"><em style=font-size:40px;>!</em>Assessment 2 Modelling etc. Starts from here


## Import Data
Here you have the option to just start from the outputed file from the wrangling if that has been ran before. 
Or can go e2e

I think ideally we would want to move from output to a new input folder somewhere on HDFS

In [67]:
//use the set wrangled from above
var df_clean = df_not_null

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


### Load from saved local file

ctrl + / or cmd + / should can comment / uncomment selections

In [15]:
! hadoop fs -mkdir -p  /tmp/output
! hadoop fs -put   -p  ./../data-clean/cleanMelbourneData.csv  /tmp/output  

put: `/tmp/output/cleanMelbourneData.csv': File exists




In [16]:
// Load Clean Dataset into a DataFrame from HDFS after wrangling is completed
var df_clean = spark
    .read
    .format("csv")
    .option("header", "true")
    .load("hdfs://localhost:9000/tmp/input/*.csv")
df_clean.cache()

//when we use the output from the wrangle 
//this can be removed as types should be fine
df_clean = df_clean.withColumn("Price",col("Price").cast("Double"))
    .withColumn("Rooms",col("Rooms").cast("Int"))
    .withColumn("DistanceFromCBD",col("DistanceFromCBD").cast("Double"))
    .withColumn("MethodOfSale",col("MethodOfSale").cast("Int"))
    .withColumn("PropertyType",col("PropertyType").cast("Int"))
    .withColumn("Bathroom",col("Bathroom").cast("Int"))
    .withColumn("Car",col("Car").cast("Int"))
    .withColumn("Landsize",col("Landsize").cast("Double"))
    .withColumn("Latitude",col("Latitude").cast("Double"))
    .withColumn("Longtitude",col("Longtitude").cast("Double"))


2020-06-07 06:32:26,284 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.


df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]
df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [17]:
//debug check file load
df_clean.cache()
df_clean.printSchema()
df_clean.count()

2020-06-07 06:32:26,523 WARN  [Thread-4] execution.CacheManager (Logging.scala:logWarning(66)) - Asked to cache already cached data.
root
 |-- Price: double (nullable = true)
 |-- MethodOfSale: integer (nullable = true)
 |-- PropertyType: integer (nullable = true)
 |-- DistanceFromCBD: double (nullable = true)
 |-- Rooms: integer (nullable = true)
 |-- Bathroom: integer (nullable = true)
 |-- Car: integer (nullable = true)
 |-- Landsize: double (nullable = true)
 |-- Latitude: double (nullable = true)
 |-- Longtitude: double (nullable = true)
 |-- Suburb: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- StreetName: string (nullable = true)



res8: Long = 15728


 ### Check Spark Parameters

In [18]:
import org.apache.spark.sql.SparkSession
import org.apache.spark.{SparkConf,SparkContext}

val cs = spark.sparkContext.getConf
sc.getConf.getAll.foreach { println }

(spark.driver.host,a56a95a92af2)
(spark.driver.port,36855)
(spark.repl.class.outputDir,/tmp/tmpd3lhnv14)
(spark.rdd.compress,True)
(spark.serializer.objectStreamReset,100)
(spark.repl.class.uri,spark://a56a95a92af2:36855/classes)
(spark.master,local[*])
(spark.executor.id,driver)
(spark.submit.deployMode,client)
(spark.app.id,local-1591511179864)
(spark.app.name,spylon-kernel)
(spark.ui.showConsoleProgress,true)


import org.apache.spark.sql.SparkSession
import org.apache.spark.{SparkConf, SparkContext}
cs: org.apache.spark.SparkConf = org.apache.spark.SparkConf@7909f61d


### Construct vectors from attributes
#### Transform Sale Date into a numeric value

In [19]:
//does this don't do much for Pat's LR model
//so i removed it, but i have left it here incase others used it
df_clean = df_clean.withColumn("Date",unix_timestamp($"Date", "dd/mm/yyyy"))

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 11 more fields]


In [20]:
import org.apache.spark.ml.feature.{FeatureHasher,OneHotEncoder,VectorAssembler}

//PK i have refactored the separte instances of hashers, encoders and assemblers to just using the feature hasher
//this will handle all the categorical items and and one hot encoding in single call

//set the feature names
val featureColumnNames= Array("MethodOfSale",
            "PropertyType",
            "DistanceFromCBD",
            "Rooms",
            "Bathroom",
            "Car",
            "Landsize",
            "Latitude",
            "Longtitude",
            "Suburb",
            "Date",
            "StreetName")


//set the categorical names
val categoricalFeatureColumnNames= Array("MethodOfSale",
            "PropertyType",
            "Suburb",
            "StreetName")

//define hasher
val hasher = new FeatureHasher()
  .setInputCols(featureColumnNames)
  .setCategoricalCols(categoricalFeatureColumnNames)
  .setOutputCol("hashedFeatures")


import org.apache.spark.ml.feature.{FeatureHasher, OneHotEncoder, VectorAssembler}
featureColumnNames: Array[String] = Array(MethodOfSale, PropertyType, DistanceFromCBD, Rooms, Bathroom, Car, Landsize, Latitude, Longtitude, Suburb, Date, StreetName)
categoricalFeatureColumnNames: Array[String] = Array(MethodOfSale, PropertyType, Suburb, StreetName)
hasher: org.apache.spark.ml.feature.FeatureHasher = featureHasher_6ae964db2281


### Apply Log transformation to price

In [21]:
df_clean = df_clean.withColumn("loggedPrice", log($"Price"))

df_clean: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 12 more fields]


In [22]:
df_clean.select("loggedPrice").show(4)

+------------------+
|       loggedPrice|
+------------------+
|14.207552645740298|
|13.849911984681606|
|14.197365800433305|
|13.652991628466498|
+------------------+
only showing top 4 rows



In [23]:
//check the antilog - introduces a slight rounding error
//could fix in future
df_clean.withColumn("PriceAntilog", exp((col("loggedPrice")))).select("Price","PriceAntilog").show(10)

+---------+------------------+
|    Price|      PriceAntilog|
+---------+------------------+
|1480000.0|1480000.0000000007|
|1035000.0|1034999.9999999997|
|1465000.0|1465000.0000000005|
| 850000.0| 849999.9999999994|
|1600000.0| 1600000.000000001|
| 941000.0| 940999.9999999994|
|1876000.0|1876000.0000000012|
|1636000.0| 1635999.999999999|
|1097000.0|1096999.9999999995|
|1350000.0|1350000.0000000002|
+---------+------------------+
only showing top 10 rows



### Split Data into a Training and a Testing Set

In [24]:
//define function to take a data frame and set training and test samples
//stratafied sample on propertype type
//original random sample may have been skewing the model
//used propertype as the prices vary wildly based on the type 

//idea is split into training and testing - then stratify the training set so its balanced.

//https://maet3608.github.io/nuts-ml/tutorial/split_stratify.html#id1

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._


def train_test_split(data: DataFrame) = {
    
    //split into 80% 20%
    var Array(train, test) = data.randomSplit(Array(0.8, 0.2), seed = 30)
    
    //Stratified sampler
    //want to over sample the other propertyTypes
    val fractions = Map(1 -> 0.8,2 ->0.9, 3 -> 0.9)

     train = train.stat.sampleBy("PropertyType",fractions,36L)
    
    (train, test)
}

val (train, test) = train_test_split(df_clean)

train.cache()
test.cache()

//debug sample counts
//println(train.count())
//println(test.count())

import org.apache.spark.sql.DataFrame
import org.apache.spark.sql.functions._
train_test_split: (data: org.apache.spark.sql.DataFrame)(org.apache.spark.sql.Dataset[org.apache.spark.sql.Row], org.apache.spark.sql.Dataset[org.apache.spark.sql.Row])
train: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 12 more fields]
test: org.apache.spark.sql.Dataset[org.apache.spark.sql.Row] = [Price: double, MethodOfSale: int ... 12 more fields]
res12: test.type = [Price: double, MethodOfSale: int ... 12 more fields]


### 1. Apply Linear Regression


###  Useful Specs for this section 

Feel free to move to references i guess

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.regression.LinearRegression

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.tuning.ParamGridBuilder

https://spark.apache.org/docs/latest/api/scala/index.html#org.apache.spark.ml.feature.FeatureHasher

https://spark.apache.org/docs/latest/ml-classification-regression.html#linear-regression

https://spark.apache.org/docs/latest/mllib-evaluation-metrics.html#regression-model-evaluation

https://spark.apache.org/docs/1.6.2/api/java/org/apache/spark/sql/DataFrameStatFunctions.html#sampleBy(java.lang.String,%20java.util.Map,%20long)








In [25]:
//imports for linear regression code in one spot
//to be moved with other imports to start

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.Pipeline

import org.apache.spark.ml.regression.LinearRegression
import org.apache.spark.ml.evaluation.RegressionEvaluator
import org.apache.spark.ml.tuning.{CrossValidator, ParamGridBuilder}
import org.apache.spark.ml.param.ParamMap
import org.apache.spark.ml.Pipeline


In [26]:
//function for getting execution time from start and end times
def getExecutionTime(start: Long , end : Long) = {
    val duration:Long = (end - start) / 1000
    (duration)
}

getExecutionTime: (start: Long, end: Long)Long


### Run first LR model with default Params
Assess the training set

In [27]:
//Assess the above LR
val startTimeMillis = System.currentTimeMillis()

//just give a LR a go with default settings values
val lr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted loggedPrice")
    .setMaxIter(1500)

//make a dataset for testing the model and printing its summarry
val featurized = hasher.transform(train)
val lrModel = lr.fit(featurized)

// Print the coefficients and intercept for linear regression
println(s"Coefficients: ${lrModel.coefficients} Intercept: ${lrModel.intercept}")

// Summarize the model over the training set and print out some metrics
val trainingSummary = lrModel.summary

println("\n")

println(s"numIterations: ${trainingSummary.totalIterations}")
println(s"objectiveHistory: [${trainingSummary.objectiveHistory.mkString(",")}]")
println("\n")

trainingSummary.residuals.show()

println("\n")
println(s"RMSE: ${trainingSummary.rootMeanSquaredError}")
println(s"r2: ${trainingSummary.r2}")
println("\n")

//print runtime
val endTimeMillis = System.currentTimeMillis()

print("Model was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


2020-06-07 06:32:33,248 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeSystemBLAS
2020-06-07 06:32:33,249 WARN  [Thread-4] netlib.BLAS (BLAS.java:<clinit>(61)) - Failed to load implementation from: com.github.fommil.netlib.NativeRefBLAS


Coefficients: (262144,[23,38,54,85,200,209,238,240,246,403,441,457,507,508,510,562,589,603,629,651,706,709,845,894,943,950,1004,1025,1031,1059,1179,1249,1272,1340,1364,1372,1438,1512,1603,1750,1764,1815,1819,1867,1874,1972,2065,2079,2090,2158,2242,2284,2307,2354,2389,2398,2467,2476,2620,2630,2669,2695,2702,2760,2764,2807,2835,2933,2940,3036,3093,3101,3161,3187,3208,3298,3351,3380,3447,3556,3640,3667,3677,3735,3748,3817,3846,3849,3894,3963,3972,3987,4080,4093,4127,4157,4160,4223,4266,4278,4345,4381,4396,4397,4411,4447,4462,4496,4549,4566,4614,4652,4694,4780,4804,4843,4853,4878,4890,4914,4917,4960,5030,5125,5147,5154,5160,5166,5177,5196,5409,5417,5442,5510,5516,5540,5569,5632,5646,5700,5753,5777,5852,6144,6186,6204,6239,6349,6366,6453,6463,6484,6503,6509,6512,6637,6645,6701,6732,6756,6904,6943,6947,6951,6983,6998,7013,7116,7145,7210,7248,7265,7326,7329,7333,7369,7385,7393,7408,7458,7560,7564,7591,7612,7617,7634,7645,7665,7783,7785,7903,7916,7918,7968,7995,8039,8135,8154,8159,8177,8189,82

7845,127912,127951,127962,127971,127998,128024,128028,128109,128179,128214,128260,128290,128293,128338,128350,128371,128372,128373,128390,128442,128453,128559,128564,128770,128803,128884,128970,129035,129067,129071,129102,129105,129203,129336,129546,129559,129560,129811,129859,129938,129999,130078,130093,130147,130148,130173,130186,130230,130259,130286,130381,130423,130507,130510,130514,130521,130581,130688,130723,130785,130819,130877,130897,130903,130951,131012,131028,131101,131104,131229,131288,131457,131491,131516,131536,131566,131710,131722,131724,131736,131747,131795,131828,131850,131890,131905,131947,131965,131997,132007,132026,132102,132192,132275,132329,132335,132351,132370,132396,132437,132457,132528,132553,132586,132610,132668,132671,132689,132728,132776,132808,132882,132885,132902,132943,132985,132988,133007,133089,133091,133112,133199,133216,133218,133253,133254,133344,133361,133384,133391,133444,133462,133464,133469,133500,133526,133548,133598,133605,133877,133897,133956,1

,239452,239498,239501,239509,239519,239586,239590,239681,239727,239770,239886,239892,239936,240028,240054,240190,240218,240301,240325,240386,240419,240434,240468,240616,240632,240633,240661,240684,240727,240732,240749,240841,240883,240906,240912,240922,241010,241108,241246,241254,241255,241301,241374,241432,241480,241492,241572,241588,241611,241687,241728,241753,241788,241863,241889,241964,241995,242079,242089,242106,242126,242127,242129,242156,242222,242244,242252,242282,242283,242328,242340,242343,242367,242399,242427,242470,242508,242514,242557,242589,242713,242819,242852,243009,243060,243074,243105,243126,243133,243162,243164,243222,243235,243244,243367,243405,243472,243522,243534,243545,243793,243799,243801,243810,243839,243843,243860,243965,243976,243983,243992,244051,244056,244072,244103,244123,244140,244154,244200,244207,244252,244328,244338,244352,244357,244389,244431,244449,244460,244468,244479,244517,244561,244574,244577,244581,244583,244622,244627,244639,244642,244691,24478

843429585,0.2436920042241641,0.08565771248146549,-0.36141817935519516,-0.12936352474152105,-0.37571717462905235,-0.1802780732035565,-0.11515137937799053,0.0059599408085807565,0.020018311106202998,-0.13798126287725634,-0.16049332719724355,-0.09803331274278712,-0.05615736058840277,-0.0183801164097689,-0.04411057204599402,0.47681061847998807,-0.12165205028657693,0.27910629112808066,0.10483517107303063,-0.344800489234554,-0.15179967976635397,0.007325956681178718,-0.08440592587022525,-0.016466398009353,-0.29278779824533346,0.10472053508857544,-0.28937247874472144,-0.11098723448429827,-0.21270377293207382,0.0740862250086087,-0.08463867729042907,-0.04462185919689259,0.4637949792344696,0.1374547824490573,-0.010587586345373975,-0.037089614675094314,-0.09281320620677762,-0.29923013780192503,0.22204018755139965,0.10815925151164991,-0.2576048770833142,-0.10130678602930021,-0.3486322574425362,-0.3558827902437681,-0.060601233118186056,0.09392597896830403,0.19324713838301125,-0.19853931342992318,-0.0

...

.045387021992083464,0.02888983111315709,-0.6607442450907518,-0.12999175147019737,-0.004448195724849106,-0.07640540794940061,0.1513802072656892,-0.009268944316726192,-0.21456232746636317,0.05236078586161083,-0.5358031845359826,-0.1349849812701786,-0.009649117384165278,-0.15670719237061667,-0.07354394758364077,0.18928065374375963,0.11701089280947372,-0.06154001277643178,-0.10113000542914702,-0.24071160970742791,-0.3900759738720335,-0.17719502635941548,-0.08642238989423526,-0.15932729103138582,-0.09401828280052875,-0.3124929287085265,-0.047977176350779145,0.35881886364036564,-0.06557727892194112,0.21485121012595038,-0.005036109245115474,-0.22243781796048145,0.18815160837491066,0.17653046104186443,0.073563486941976,-0.20347686521203412,-0.12460480509242514,0.04903152051008303,-0.14365504405357005,-0.25354427854168265,-0.26368436805996254,-0.1019603226217504,0.09803689925271011,-0.20785424059022292,-0.1827249476390462,-0.1280240588022967,0.3702029194356996,0.1467101467020101,-0.043829423186

6983,-0.17481678558894168,-0.2738015979746647,0.1794492259892374,0.025212428235614,-0.39487213762848095,-0.033688605732268846,-0.12614347342715815,-0.16937621639427047,-0.014281252821355752,-0.3023004240574374,-0.20343032658857368,-0.05633788326333389,0.008962564615446197,-0.08645198188161754,-0.4692099791727071,0.0443877668856402,0.08992485146372198,0.04418685939418966,-0.30640614925452175,0.4603704432702399,0.8294485172268498,-0.4284622391589497,-0.1741985023582529,0.08805259500875931,-0.2982317935873966,0.1262873121458701,-0.47741505718399235,-0.7470738970720104,0.2681042856593802,-0.11516450259132158,-0.04175071471939835,-0.4974945772772317,0.2924215510270909,-0.39100521465926025,-0.19677480809286238,0.15303458442522988,-0.07379996641084242,0.36505764456941975,-0.0012520015441809173,0.1822592807061415,0.2376265467688514,-0.156245014163958,-0.07334868443436224,-0.1539205935758793,-0.19986438507306098,-0.04408251974612408,0.06554372272927453,-0.3602940173977117,0.16038039359742257,0.

336,-0.010378616133178335,0.012832837696711934,-0.3939515992172239,-0.1785102663257592,0.0832801793152071,0.053348828649958026,0.08914169164143454,0.08663334653487709,-0.13245503856424692,-0.021813755768145624,0.37929377087544497,0.04221146496780787,-0.3058893931227382,-0.08279103092854942,-0.1042555496997849,-0.3858526009519956,-0.5319560362134872,0.011600311727788234,0.025396913145675437,-0.06980969950213899,0.0150329456000135,0.0548742893979068,0.1276836192087275,0.07637432967328846,-0.022316941971262674,-0.6958842850553508,-0.06430307741262266,-0.2698353323520038,-0.1226842795336895,-0.1379561149562494,-0.015169642210272623,-0.2088883626707806,-0.15864901447418664,0.29413079003497583,-0.0924271800508458,0.5189286357768162,0.2443928009320659,0.3727511116118977,-0.5721215700278534,-0.24557416967648837,-0.1878535335163398,0.1488825251016236,0.13396917839156064,-0.12991609344134233,0.28333787232352275,-0.2176337191470485,-0.08717869132672627,0.09383355078828372,-0.2147660976598643,-0.1

2498266611138493,-0.356618966027567,-0.26451591404211494,0.08603217204216837,-0.2487892122424819,-0.1971957975826874,-0.0676096757156551,0.08846884589768521,0.04319511070335615,0.003367846558089315,-0.022571148355747347,-0.0856006857307671,-0.22249873217610303,0.12104899075693931,-0.3673535222959606,-0.1940179414565829,-0.28673690598305734,0.25534805861023263,0.33483528528983175,0.06073176489142836,0.0376821175633071,-0.1305309826931595,-0.2661049852006236,0.09893237585760023,0.031822258076906744,-0.09385278271033773,0.09817464200955348,0.009841462389934563,0.08196837199653999,0.12213930632275236,-0.1180537097604254,-0.18081473907672369,-0.06853689072007078,0.4946794804782336,-0.2641892180967004,0.3743892983807374,-0.3411148767259209,0.08016285089256774,0.5896474196416258,0.10091695923650733,0.021704303991862878,0.21072148602735266,-0.22552601845025583]) Intercept: 118.05392267053989


numIterations: 150
objectiveHistory: [0.4999513287258013,0.35212982889237715,0.06944644752939652,0.06

startTimeMillis: Long = 1591511550775
lr: org.apache.spark.ml.regression.LinearRegression = linReg_b24b23450960
featurized: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
lrModel: org.apache.spark.ml.regression.LinearRegressionModel = linReg_b24b23450960
trainingSummary: org.apache.spark.ml.regression.LinearRegressionTrainingSummary = org.apache.spark.ml.regression.LinearRegressionTrainingSummary@75c14887
endTimeMillis: Long = 1591511571757


### Run Crossvalidator on LR model
Construct paramgrid of regParam and ElasticNet

Estimate the performance using RMSE

Get best Params to apply in final pipeline

One could add this to the pipepline but this takes the longest to run
as its assessing 4*4 combinations - so instead the learnings of this are passed to pipeline
Then this wouldn't need to run

<span style="color:red"><em style=font-size:40px;>!</em>Runtime of next cell is around 250 seconds or so, on my VM with like 10gig of RAM and 4 CPUs</span>



In [28]:
//Creates a crossvalidator on only the LR model
//Had some issues retreiving the params when using on pipeline
//and the pipeline only has one estimator in it so this was easier than traversing the stages of the pipline

val startTimeMillis = System.currentTimeMillis()

// make some featurised sets for just running the model thru the Crossvalidator
val featurized_training = hasher.transform(train)
val featurized_test = hasher.transform(test)

//set LR with 1500 max iter
val lr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted LoggedPrice")
    .setMaxIter(1500)


// We use a ParamGridBuilder to construct a grid of parameters to search over.
val paramGrid = new ParamGridBuilder()
  .addGrid(lr.regParam, Array(0,0.1,0.5,1))
  .addGrid(lr.elasticNetParam, Array(0,0.1,0.5,1))
  .build()

// We now treat the model as an Estimator, wrapping it in a CrossValidator instance.
// This will allow us to choose best params for the model
val cv = new CrossValidator()
  .setEstimator(lr)
  .setEvaluator(new RegressionEvaluator()
  .setLabelCol("loggedPrice")
  .setPredictionCol("Predicted LoggedPrice")
  .setMetricName("rmse"))
  .setEstimatorParamMaps(paramGrid)
  .setNumFolds(5)  
  .setParallelism(2)

// Run cross-validation, and choose the best set of parameters.
val cvModel = cv.fit(featurized_training)

// Make predictions on test documents. 
//cvModel uses the best model found.
cvModel.transform(featurized_test)
  .select("loggedPrice", "Predicted LoggedPrice")
  .show()

//print runtime
val endTimeMillis = System.currentTimeMillis()

print("Model was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


2020-06-07 06:34:45,383 WARN  [block-manager-slave-async-thread-pool-47] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_4613, which does not exist
2020-06-07 06:35:06,563 WARN  [block-manager-slave-async-thread-pool-61] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_5674, which does not exist
2020-06-07 06:35:06,563 WARN  [block-manager-slave-async-thread-pool-61] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_5674_piece0, which does not exist
2020-06-07 06:36:17,850 WARN  [block-manager-slave-async-thread-pool-76] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_8587, which does not exist
2020-06-07 06:36:18,571 WARN  [block-manager-slave-async-thread-pool-132] storage.BlockManager (Logging.scala:logWarning(66)) - Asked to remove block broadcast_8629, which does not exist
+------------------+---------------------+
|       loggedPrice

startTimeMillis: Long = 1591511572229
featurized_training: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
featurized_test: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 13 more fields]
lr: org.apache.spark.ml.regression.LinearRegression = linReg_863e887570af
paramGrid: Array[org.apache.spark.ml.param.ParamMap] =
Array({
	linReg_863e887570af-elasticNetParam: 0.0,
	linReg_863e887570af-regParam: 0.0
}, {
	linReg_863e887570af-elasticNetParam: 0.1,
	linReg_863e887570af-regParam: 0.0
}, {
	linReg_863e887570af-elasticNetParam: 0.5,
	linReg_863e887570af-regParam: 0.0
}, {
	linReg_863e887570af-elasticNetParam: 1.0,
	linReg_863e887570af-regParam: 0.0
}, {
	linReg_863e887570af-elasticNetParam: 0.0,
	linReg_863e887570af-regParam: 0.1
...

### Extract best Parameters from CrossValidation

In [30]:
//print out the params used for the best model
val bestModel = cvModel.bestModel

//save as ParamMap to pass into pipeline
val bestParamMap = bestModel.extractParamMap

//these the bestParams determined from CV
println(bestModel.extractParamMap) 

{
	linReg_863e887570af-aggregationDepth: 2,
	linReg_863e887570af-elasticNetParam: 0.0,
	linReg_863e887570af-epsilon: 1.35,
	linReg_863e887570af-featuresCol: hashedFeatures,
	linReg_863e887570af-fitIntercept: true,
	linReg_863e887570af-labelCol: loggedPrice,
	linReg_863e887570af-loss: squaredError,
	linReg_863e887570af-maxIter: 1500,
	linReg_863e887570af-predictionCol: Predicted LoggedPrice,
	linReg_863e887570af-regParam: 0.1,
	linReg_863e887570af-solver: auto,
	linReg_863e887570af-standardization: true,
	linReg_863e887570af-tol: 1.0E-6
}


bestModel: org.apache.spark.ml.Model[_] = linReg_863e887570af
bestParamMap: org.apache.spark.ml.param.ParamMap =
{
	linReg_863e887570af-aggregationDepth: 2,
	linReg_863e887570af-elasticNetParam: 0.0,
	linReg_863e887570af-epsilon: 1.35,
	linReg_863e887570af-featuresCol: hashedFeatures,
	linReg_863e887570af-fitIntercept: true,
	linReg_863e887570af-labelCol: loggedPrice,
	linReg_863e887570af-loss: squaredError,
	linReg_863e887570af-maxIter: 1500,
	linReg_863e887570af-predictionCol: Predicted LoggedPrice,
	linReg_863e887570af-regParam: 0.1,
	linReg_863e887570af-solver: auto,
	linReg_863e887570af-standardization: true,
	linReg_863e887570af-tol: 1.0E-6
}


In [31]:
//define new LR instance for using the best params in pipeline
val bestlr = new LinearRegression()
    .setLabelCol("loggedPrice")
    .setFeaturesCol("hashedFeatures")
    .setPredictionCol("Predicted loggedPrice")

bestlr: org.apache.spark.ml.regression.LinearRegression = linReg_0946d4825601


### Define Pipeline 

In [32]:
// add linear regression to stages
// I removed the scaler as the performance for gain was terrible
val lrStages = Array(
    hasher,
    bestlr
)

lrStages: Array[org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable{def copy(extra: org.apache.spark.ml.param.ParamMap): org.apache.spark.ml.PipelineStage with org.apache.spark.ml.util.DefaultParamsWritable}}] = Array(featureHasher_6ae964db2281, linReg_0946d4825601)


In [33]:
val startTimeMillis = System.currentTimeMillis()

//define a pipleine
val lrPipe = new Pipeline().setStages(lrStages)

// We fit our DataFrame into the pipeline to generate a model
// pass best ParamMap from cross validation
val lrModel = lrPipe.fit(train,bestParamMap)


// Make predictions using the model and the test data
// pass best ParamMap from cross validation
val predictions = lrModel.transform(test,bestParamMap)

val endTimeMillis = System.currentTimeMillis()

print("Pipeline was executed "
      + getExecutionTime(startTimeMillis,endTimeMillis))


Pipeline was executed 6

startTimeMillis: Long = 1591511844591
lrPipe: org.apache.spark.ml.Pipeline = pipeline_2d6dc4534246
lrModel: org.apache.spark.ml.PipelineModel = pipeline_2d6dc4534246
predictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 14 more fields]
endTimeMillis: Long = 1591511851038


In [34]:
def getExecutionTime(start: Long , end : Long) = {
    val duration:Long = (end - start) / 1000
    (duration)
}

getExecutionTime: (start: Long, end: Long)Long


### Undo the Log transformation  on the prediciton

In [35]:
var Antilogpredictions = predictions.withColumn("Predicted loggedPrice",col("Predicted loggedPrice").cast("Double"))
Antilogpredictions.withColumn("Predicted Price", exp((col("Predicted loggedPrice")))).select("Predicted Price").show()

+------------------+
|   Predicted Price|
+------------------+
| 263181.8243934773|
| 328613.8383136014|
| 570534.5417677251|
|334065.07986165537|
| 567523.5464109678|
|358916.51907410205|
| 564747.2064573611|
|   635731.89636228|
| 547555.5627298004|
|397927.73849305813|
| 513546.6213028271|
|351860.53679235553|
| 421850.3942012743|
| 507467.7724472131|
|346491.18381457595|
|307809.27769534563|
| 535888.2521617918|
|  646813.369725167|
| 341684.6959471435|
| 319123.2910207064|
+------------------+
only showing top 20 rows



Antilogpredictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 14 more fields]


In [36]:
Antilogpredictions = Antilogpredictions.withColumn("Predicted Price", exp((col("Predicted loggedPrice"))))

Antilogpredictions: org.apache.spark.sql.DataFrame = [Price: double, MethodOfSale: int ... 15 more fields]


### Evaluation of the model and its predictions

In [37]:
//finesse the output of predicted price and price to aid visual compare
Antilogpredictions.withColumn("Predicted Price", round($"Predicted Price", 0))
    .select("Price","Predicted Price").show()

+--------+---------------+
|   Price|Predicted Price|
+--------+---------------+
|170000.0|       263182.0|
|280000.0|       328614.0|
|280500.0|       570535.0|
|283000.0|       334065.0|
|290000.0|       567524.0|
|300000.0|       358917.0|
|300000.0|       564747.0|
|305000.0|       635732.0|
|310000.0|       547556.0|
|316000.0|       397928.0|
|320000.0|       513547.0|
|320000.0|       351861.0|
|320000.0|       421850.0|
|320000.0|       507468.0|
|325000.0|       346491.0|
|333000.0|       307809.0|
|340000.0|       535888.0|
|345000.0|       646813.0|
|348000.0|       341685.0|
|350000.0|       319123.0|
+--------+---------------+
only showing top 20 rows



In [38]:
Antilogpredictions.describe().select("summary","Price","Predicted Price").show()

+-------+-----------------+------------------+
|summary|            Price|   Predicted Price|
+-------+-----------------+------------------+
|  count|             3108|              3108|
|   mean|1148713.091055341|1144555.5274824973|
| stddev|644982.7776402166| 574733.7751450145|
|    min|         170000.0|192204.77409173994|
|    max|        6250000.0| 7627710.166948065|
+-------+-----------------+------------------+



In [39]:
// define an evaluator for the cross validation
def evaluate ( predictions: DataFrame, metric: String) = {
    val eval =  new RegressionEvaluator()
       .setLabelCol("loggedPrice")
       .setPredictionCol("Predicted loggedPrice")
       .setMetricName(metric)
println("Root Mean Squared Error "+  metric.toUpperCase()
        +" on test data = " + eval.evaluate(predictions))
}

evaluate: (predictions: org.apache.spark.sql.DataFrame, metric: String)Unit


#### Regression metrics

**Mean squared error (MSE)** -- the average of squared differences between the predicted outcome and the true outcome.

**R2 coefficient** -- the proportion of variance in the outcome that our model is capable of predicting based on its features.

In [40]:
//difference between logged price and predicted logged price
evaluate(Antilogpredictions,"rmse")
evaluate(Antilogpredictions,"r2")

Root Mean Squared Error RMSE on test data = 0.2566778441301778
Root Mean Squared Error R2 on test data = 0.7241766708776317


Looks like we go down from our training set:

`RMSE: 0.15067909288639053
r2: 0.9080852142542626`

to our test set with best params:

`0.2566778441301778
0.7241766708776317`


### 2. Apply K-means Model  

This modelling of this section is based on (Sarkar, 2017)

#### Training


Pipeline Estimator

In [45]:
import org.apache.spark.ml.clustering.KMeans

val kmeans = new KMeans().setK(2).setSeed(1L)
val model = kmeans.fit(df)

<console>: 39: error: not found: value df

#### Prediction

#### Testing/Evaluation

Pipeline Model Transformer

Evaluate the quality of clustering by "elbowing" the **Within SEt Sum of Squared Errors (WSSSE)** graph.

In [ ]:
val WSSSE = model.computeCost(df)

println("WSSSE error + $WSSSE")

In [ ]:
model.clusterCenters.foreach(println)

In [ ]:
val transformed = model.transform(df)

In [ ]:
//compute the diff between the labels abd the predicated values on data set

transformed.select("prediction").groupBy("prediction")count().orderBy().show("prediction")

In [ ]:
val y1df = transformed.select($"label",$"prediction").where($"label!=prediction")

In [ ]:
y1df.count()

In [ ]:
transformed.filter("prediction = 0").show()

In [ ]:
transformed.filter("prediction = 1").show()

In [ ]:
transformed.filter("prediction = 0").describe.show()

In [ ]:
transformed.filter("prediction = 1").describe.show()

In [ ]:
println("No of mis-matche between predictions and labels ="+y1df.count() +"\nTotal no. of records "+transformed.count()+ \
         "\nCorrect predictions = "+ (1-y1df.count()).toDouble/transformed.count()+"\nMismatch "+\
          (1-y1df.count()).toDouble/transformed.count())

In [ ]:
//Feed test input records for the model to predict their cluster
val testData = spark.createDataFrame(Seq().toDF("colName","features"))
model.transform(testData).show()

### 3. Apply Random Forest Regression

**Build Random Forest model**
Specify maxDepth, maxBins, auto and seed parameters.

**maxDepth** -- Maximum depth of a tree. Increasing the depth makes the model more powerful, but deep trees take longer to train.

**maxBins** -- Maximum number of bins used for discretizing continuous features and for choosing how to split on features at each node.

**auto** -- Automatically select the number of features to consider for splits at each tree node

**seed** -- Use a random seed number , allowing to repeat the results


If the number of trees is 1, then no bootstrapping is used at all. However, if the number of trees is > 1, then the bootstrapping is accomplished. Where, the parameter featureSubsetStrategy signifies the number of features to be considered for splits at each node. The supported values of featureSubsetStrategy are "auto", "all", "sqrt", "log2" and "on third". The supported numerical values, on the other hand, are (0.0-1.0] and [1-n]. However, if featureSubsetStrategy is chosen as "auto", the algorithm chooses the best feature subset strategy automatically


If the numTrees == 1, the featureSubsetStrategy is set to be "all". However, if the numTrees > 1 (i.e., forest), featureSubsetStrategy is set to be "onethird" for regression


Moreover, if a real value "n" is in the range (0, 1.0] is set, n*number_of_features is used consequently. However, if an integer value "n" is in the range (1, the number of features) is set, only n features are used alternatively


The parameter categoricalFeaturesInfo which is a map is used for storing arbitrary of categorical features. An entry (n -> k) indicates that feature n is categorical with k categories indexed from 0: {0, 1,...,k-1}
The impurity criterion used for information gain calculation. The supported values are “gini" and “variance”. The former is the only supported value for classification. The latter is used for regression


The maxDepth is the maximum depth of the tree. (e.g., depth 0 means 1 leaf node, depth 1 means 1 internal node + 2 leaf nodes). However, the suggested value is 4 to get a better result


The maxBins signifies the maximum number of bins used for splitting the features; where the suggested value is 100 to get better results


Finally, the random seed is used for bootstrapping and choosing feature subsets to avoid the random nature of the results.

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor
import org.apache.spark.ml.tuning.CrossValidator
//import org.apache.spark.ml.Pipeline

val seed = 5043

val rf = new RandomForestRegressor()
  .setMaxBins(100)
  .setMaxDepth(6)
  .setNumTrees(10)
  .setFeatureSubsetStrategy("onethird")
  .setSeed(seed)
  .setLabelCol("Price")

In [ ]:
val rfPredictions = time{predictions(rf, train, test)}
rfPredictions.cache()

In [ ]:
rfPredictions.columns

In [ ]:
rfPredictions.withColumn("prediction", round($"prediction", 0)).select("Price","prediction").show()

#### Regression metrics


In [ ]:
evaluate(rfPredictions,"rmse")

In [ ]:
evaluate(rfPredictions,"r2")

#### Testing/Evaluation/ Parameter Tuning

Cross-validation
<span style="color:red">
TO DO: 
* finish implementation for Cross-validation 
* check if finish run in reasonable time
</span>

In [ ]:
import org.apache.spark.ml.regression.RandomForestRegressor

// Models hypoparameters
val numTrees = Seq(5,10,30)
val maxBins = Seq(50,100)
val maxDepth = Seq(2,3,5)
val impurity = Seq("gini","entropy","variance")
val featureSubsetStrategy = Seq("sqrt","onethird")

val rf = new RandomForestRegressor()
  .setLabelCol("Price")
  .setFeaturesCol("features")
  .setPredictionCol("prediction")


val rfParamMap = new ParamGridBuilder()
  .addGrid(rf.numTrees, numTrees)
  .addGrid(rf.maxDepth, maxDepth)
  .addGrid(rf.maxBins, maxBins)
  .addGrid(rf.featureSubsetStrategy, featureSubsetStrategy)
  .build()

val t0 = System.nanoTime()
val best_model = train_eval(rf, rfParamMap, train, test)
val t1 = System.nanoTime()
println("Elapsed time: " + (t1 - t0)/(1000000000) + " s")


#### Prediction

In [ ]:
rfPredictions.withColumn("prediction", round($"prediction", 0)).select("Price","prediction").show()

// this will add new columns rawPrediction, probability and prediction
val predictionDf = randomForestModel.transform(testData)
predictionDf.show(10)

#### Tuning

#### Bias vs Variance Graph of Error (validation error and training error) versus training set size. 


<span style="color:red">
TO DO: 
produce graph -- validation error and training error should converge
</span>


### References

Apache Spark (n.d.). Spark ML Programming Guide. Retrieved from https://spark.apache.org/docs/1.2.2/ml-guide.html

Gorczynski M. (2017). Introduction to machine learning with spark and mllib (dataframe API). Retrieved from https://scalac.io/scala-spark-ml-machine-learning-introduction/

Hydrospheredata (2020). Program creek. Scala Code Examples. Scaler Retrieved from https://www.programcreek.com/scala/org.apache.spark.ml.feature.StandardScaler

Jen G. (2020) FeatureHasher. Retrieved from https://george-jen.gitbook.io/data-science-and-apache-spark/featurehasher

Johnson S (2019). From sckit-learn to Spark ML. Retrieved from https://towardsdatascience.com/from-scikit-learn-to-spark-ml-f2886fb46852

Johnson S (2019). Housing Prices - Spark ML Project Retrieved from https://github.com/scottdjohnson/HousingPricePredictions/blob/master/HousingPrices-SparkML.ipynb

Masri A. (2019). FeatureTransformation. Retrieved from
https://towardsdatascience.com/apache-spark-mllib-tutorial-7aba8a1dce6e

Sarkar A. (2017). Learning Spark SQL. Implementing a Spark ML clustering model. Packt Publishing.

Scala Doc (n.d.) Retrieved from https://docs.scala-lang.org


(2019) Random Forest Classifier with Apache Spark Retireved from https://medium.com/rahasak/random-forest-classifier-with-apache-spark-c63b4a23a7cc

Wagle M.(2020) _Predicting House Prices using Machine Learning_. Retrieved from https://medium.com/@manilwagle/predicting-house-prices-using-machine-learning-cab0b82cd3f